In [116]:
import pandas as pd
import numpy as np
from dask import dataframe as dd
from sklearn.metrics.pairwise import cosine_similarity, nan_euclidean_distances
from sklearn.preprocessing import LabelEncoder, normalize
import dask.array as da

In [241]:
ddf = dd.read_parquet('../data/processed/ProcessChexpertDfToParquet/part.0.parquet')

In [163]:
df = pd.read_parquet('../data/processed/ProcessChexpertDfToParquet/part.0.parquet')

In [80]:
df_raw = ddf.copy()

In [81]:
ddf = ddf.drop(columns = ['Path'])

In [164]:
df = df.drop(columns = ['Path'])

In [165]:
# columns that need encoding
object_cols = ddf.dtypes[(ddf.dtypes == object)].index.values

In [173]:
# columns that need encoding
object_cols = df.dtypes[(df.dtypes == object)].index.values

In [166]:
LE = LabelEncoder()

In [84]:
for object_col in object_cols:
    ddf[object_col] = da.from_array(LE.fit_transform(ddf[object_col].astype(str)))

In [174]:
for object_col in object_cols:
    df[object_col] = LE.fit_transform(df[object_col].astype(str))

In [86]:
def normalize_general(ddf,columns):
    result = ddf.copy()
    for feature_name in columns:
        max_value = ddf[feature_name].max()
        min_value = ddf[feature_name].min()
        result[feature_name] = 2*(ddf[feature_name] - min_value) / (max_value - min_value) - 1
    return result

In [87]:
def normalize_chex(ddf):
    ddf = normalize_general(ddf,object_cols)
    ddf =  normalize_general(ddf,['Age'])
    return ddf

In [88]:
ddf = normalize_chex(ddf)

In [175]:
df = normalize_chex(df)

In [176]:
df.head()

,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
__null_dask_index__,,,,,,,,,,,,,,,,,,
0,-1.0,0.511111,-1.0,-1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0
1,-1.0,0.933333,-1.0,-1.0,NaN,NaN,-1.0,1.0,NaN,-1.0,-1.0,NaN,-1.0,NaN,-1.0,NaN,1.0,NaN
2,-1.0,0.844444,-1.0,-1.0,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
3,-1.0,0.844444,1.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
4,0.0,-0.088889,-1.0,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [200]:
df = ddf.compute()

In [178]:
df.head()

,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,-1.0,0.511111,-1.0,-1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0
1,-1.0,0.933333,-1.0,-1.0,NaN,NaN,-1.0,1.0,NaN,-1.0,-1.0,NaN,-1.0,NaN,-1.0,NaN,1.0,NaN
2,-1.0,0.844444,-1.0,-1.0,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
3,-1.0,0.844444,1.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
4,0.0,-0.088889,-1.0,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [90]:
df.mode(axis=0)

,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,0.0,1.0,-1.0,-1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,-1.0,-1.0,0.0,1.0,1.0,1.0,1.0


In [91]:
# Find a CXR with lots of features
ddf[ddf['Lung Lesion']==1].isna().sum(axis=1).idxmin().compute()

37959

In [142]:
assert df[df['Lung Lesion']==1].isna().sum(axis=1).idxmin() == 37959

In [94]:
ddf.loc[37959].compute()

,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
37959,0.0,0.333333,-1.0,-1.0,NaN,0.0,NaN,1.0,1.0,1.0,1.0,-1.0,-1.0,0.0,1.0,NaN,NaN,1.0


In [242]:
index_comparator = 37959

In [243]:
row_comparator_raw = ddf.loc[index_comparator]

In [277]:
row_comparator_na = ddf.loc[index_comparator].isna()

In [275]:
row_comparator_na

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
npartitions=1,,,,,,,,,,,,,,,,,,,
37959,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool
37959,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [261]:
row_comparator_na = df.loc[index_comparator].isna()

In [262]:
row_comparator_na

Sex                           False
Age                           False
Frontal/Lateral               False
AP/PA                         False
No Finding                     True
Enlarged Cardiomediastinum    False
Cardiomegaly                   True
Lung Opacity                  False
Lung Lesion                   False
Edema                         False
Consolidation                 False
Pneumonia                     False
Atelectasis                   False
Pneumothorax                  False
Pleural Effusion              False
Pleural Other                  True
Fracture                       True
Support Devices               False
Name: 37959, dtype: bool

In [149]:
row_comparator.values

array([False, False, False, False,  True, False,  True, False, False,
       False, False, False, False, False, False,  True,  True, False])

In [150]:
df[df.isna().sum(axis=1) == n_nanvalues]

,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
__null_dask_index__,,,,,,,,,,,,,,,,,,
15265,0.5,0.844444,0.0,0.75,NaN,-1.0,-1.0,1.0,NaN,0.0,0.0,NaN,-1.0,0.0,-1.0,-1.0,NaN,0.0
15266,0.5,0.844444,1.0,0.50,NaN,-1.0,-1.0,1.0,NaN,0.0,0.0,NaN,-1.0,0.0,-1.0,-1.0,NaN,0.0
19452,0.5,0.966667,0.0,0.75,NaN,-1.0,1.0,-1.0,NaN,0.0,0.0,NaN,-1.0,0.0,1.0,-1.0,1.0,NaN
19453,0.5,0.966667,1.0,0.50,NaN,-1.0,1.0,-1.0,NaN,0.0,0.0,NaN,-1.0,0.0,1.0,-1.0,1.0,NaN
37959,0.5,0.666667,0.0,0.00,NaN,0.0,NaN,1.0,1.0,1.0,1.0,-1.0,-1.0,0.0,1.0,NaN,NaN,1.0
80312,0.5,0.466667,0.0,0.75,NaN,-1.0,NaN,1.0,NaN,0.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,NaN,1.0
80313,0.5,0.466667,1.0,0.50,NaN,-1.0,NaN,1.0,NaN,0.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,NaN,1.0
116061,0.0,0.711111,0.0,0.75,NaN,0.0,0.0,1.0,0.0,0.0,0.0,NaN,1.0,0.0,-1.0,-1.0,NaN,NaN
116062,0.0,0.711111,1.0,0.50,NaN,0.0,0.0,1.0,0.0,0.0,0.0,NaN,1.0,0.0,-1.0,-1.0,NaN,NaN


In [102]:
import pandas as pd
pd.__version__

'1.1.5'

In [256]:
similar_features_idx = (ddf.isna() == row_comparator_na).sum(1)

In [257]:
similar_features_idx.compute()

0          9
1         11
2          9
3          8
4         10
          ..
223409    12
223410    12
223411     9
223412    12
223413     8
Length: 223414, dtype: int64

In [115]:
row_comparator_na.columns == ddf.isna().columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True])

In [117]:
similar_features_idx = ddf.isna().eq(row_comparator_na)

In [118]:
similar_features_idx.compute()

,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223409,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
223410,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
223411,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
223412,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [193]:
row_comparator_na = ddf.loc[index_comparator].isna().compute()

In [239]:
row_comparator_na

Sex                           False
Age                           False
Frontal/Lateral               False
AP/PA                         False
No Finding                     True
Enlarged Cardiomediastinum    False
Cardiomegaly                   True
Lung Opacity                  False
Lung Lesion                   False
Edema                         False
Consolidation                 False
Pneumonia                     False
Atelectasis                   False
Pneumothorax                  False
Pleural Effusion              False
Pleural Other                  True
Fracture                       True
Support Devices               False
Name: 37959, dtype: bool

In [240]:
similar_features_idx = (ddf.isna() == row_comparator_na).sum(1).compute().nlargest(n=100)

In [211]:
similar_features_idx = similar_features_idx.index

In [224]:
similar_features_idx.to_list()

[37959,
 4810,
 4811,
 28404,
 35737,
 35738,
 87096,
 92004,
 113518,
 132525,
 132526,
 136007,
 136008,
 145696,
 152538,
 168283,
 173799,
 195964,
 199087,
 210244,
 216617,
 219420,
 219889,
 2157,
 2600,
 2601,
 3174,
 6794,
 7986,
 8555,
 9678,
 9682,
 12419,
 13326,
 13792,
 15509,
 15962,
 15963,
 16163,
 17419,
 17421,
 18492,
 18954,
 22096,
 22097,
 22098,
 22118,
 22568,
 22570,
 22580,
 23170,
 23232,
 23233,
 23355,
 23678,
 23679,
 24581,
 24604,
 24846,
 25478,
 25984,
 33731,
 33894,
 33895,
 34363,
 34364,
 35139,
 35394,
 35565,
 35570,
 35679,
 35920,
 35921,
 36949,
 37118,
 37491,
 38293,
 38295,
 40703,
 42090,
 42091,
 42525,
 42640,
 44112,
 48915,
 53057,
 53631,
 54212,
 54867,
 55109,
 55110,
 58309,
 58368,
 58543,
 59042,
 59043,
 60157,
 60212,
 60844,
 63473]

In [219]:
row_comparator_raw.compute()

,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
37959,0.0,0.333333,-1.0,-1.0,NaN,0.0,NaN,1.0,1.0,1.0,1.0,-1.0,-1.0,0.0,1.0,NaN,NaN,1.0


In [226]:
ddf.loc[similar_features_idx.to_list()].compute()

,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
37959,0.0,0.333333,-1.0,-1.0,NaN,0.0,NaN,1.0,1.0,1.0,1.0,-1.0,-1.0,0.0,1.0,NaN,NaN,1.0
4810,-1.0,0.288889,-1.0,0.5,NaN,-1.0,NaN,1.0,1.0,0.0,0.0,NaN,-1.0,1.0,1.0,NaN,NaN,0.0
4811,-1.0,0.288889,1.0,0.0,NaN,-1.0,NaN,1.0,1.0,0.0,0.0,NaN,-1.0,1.0,1.0,NaN,NaN,0.0
28404,-1.0,-0.577778,-1.0,-1.0,NaN,-1.0,NaN,1.0,NaN,0.0,1.0,-1.0,1.0,0.0,1.0,NaN,NaN,1.0
35737,-1.0,-0.133333,-1.0,0.5,NaN,1.0,NaN,1.0,1.0,-1.0,-1.0,-1.0,-1.0,NaN,1.0,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59043,0.0,0.266667,1.0,0.0,NaN,0.0,NaN,1.0,0.0,0.0,0.0,NaN,1.0,0.0,0.0,NaN,NaN,NaN
60157,0.0,0.200000,-1.0,-1.0,NaN,0.0,NaN,1.0,NaN,1.0,-1.0,NaN,-1.0,0.0,1.0,NaN,NaN,1.0
60212,-1.0,-0.355556,-1.0,-1.0,NaN,NaN,NaN,1.0,-1.0,1.0,NaN,-1.0,1.0,1.0,1.0,NaN,NaN,1.0
60844,0.0,0.444444,-1.0,-1.0,NaN,0.0,NaN,1.0,NaN,1.0,1.0,-1.0,-1.0,NaN,-1.0,NaN,NaN,1.0


In [230]:
argsorted = nan_euclidean_distances(row_comparator_raw.compute().values.reshape(1,-1), ddf.loc[similar_features_idx.to_list()].compute().values).argsort()

In [235]:
ddf.loc[similar_features_idx[argsorted][0]].compute()

,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
37959,0.0,0.333333,-1.0,-1.0,NaN,0.0,NaN,1.0,1.0,1.0,1.0,-1.0,-1.0,0.0,1.0,NaN,NaN,1.0
17419,0.0,0.333333,-1.0,-1.0,NaN,0.0,NaN,NaN,NaN,1.0,1.0,-1.0,-1.0,0.0,1.0,NaN,NaN,1.0
173799,0.0,0.177778,-1.0,-1.0,NaN,-1.0,NaN,NaN,1.0,1.0,1.0,-1.0,-1.0,0.0,1.0,NaN,NaN,1.0
15509,0.0,0.222222,-1.0,-1.0,NaN,0.0,NaN,1.0,NaN,1.0,NaN,-1.0,-1.0,0.0,0.0,NaN,NaN,1.0
216617,-1.0,0.333333,-1.0,-1.0,NaN,-1.0,NaN,1.0,1.0,1.0,1.0,NaN,-1.0,0.0,1.0,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113518,-1.0,0.844444,-1.0,-1.0,NaN,1.0,NaN,1.0,NaN,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,NaN,NaN,1.0
33894,0.0,0.622222,-1.0,0.5,NaN,1.0,NaN,1.0,NaN,-1.0,-1.0,-1.0,-1.0,NaN,-1.0,NaN,NaN,1.0
35921,0.0,0.400000,1.0,0.0,NaN,0.0,NaN,1.0,NaN,0.0,0.0,1.0,1.0,0.0,0.0,NaN,NaN,NaN
199087,-1.0,0.911111,-1.0,-1.0,NaN,-1.0,NaN,1.0,1.0,-1.0,NaN,1.0,1.0,0.0,-1.0,NaN,NaN,1.0


In [171]:
similar_features_idx[argsorted][0]

C:\Users\wmj\.virtualenvs\radio-star-X64HrzkT\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  """Entry point for launching an IPython kernel.


array([ 37959,  17419, 160623, 189784, 144069, 173799,  15509, 197860,
       197859,  69088, 216617, 166470,  22580, 168313,  18954, 198671,
       219889,  69982, 193922,  18492, 201499, 211020, 191612,  91258,
       216463, 137647,  23170, 215838, 215835, 137648,  92004, 168283,
       184511, 213887, 213883, 100999, 114162,  54867, 154273, 153679,
       207410, 169843, 147229, 175053, 175056,   4810, 132525, 210244,
       195964,  28404,  17421, 111571,   6794,   4811, 132526, 103732,
        25478, 154482,  24581,  44112, 136007,  34363, 206976, 199849,
        87096, 188287,  42090, 219420, 136008, 212838, 216306, 170707,
       212012, 216046, 192465, 152538, 145696,  59042,  97682,  35737,
        59043,  97683,  53057, 168271, 192466,  35738, 184323, 153047,
        80312,  80313, 218450, 194979, 128621, 118165, 113518, 109698,
       113382, 138084, 113383, 199087], dtype=int64)

In [173]:
df_raw.loc[similar_features_idx[argsorted][0]]

C:\Users\wmj\.virtualenvs\radio-star-X64HrzkT\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  """Entry point for launching an IPython kernel.


,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
__null_dask_index__,,,,,,,,,,,,,,,,,,,
37959,CheXpert-v1.0-small/train/patient09313/study1/...,Male,60,Frontal,AP,NaN,0.0,NaN,1.0,1.0,1.0,1.0,-1.0,-1.0,0.0,1.0,NaN,NaN,1.0
17419,CheXpert-v1.0-small/train/patient04352/study4/...,Male,60,Frontal,AP,NaN,0.0,NaN,NaN,NaN,1.0,1.0,-1.0,-1.0,0.0,1.0,NaN,NaN,1.0
160623,CheXpert-v1.0-small/train/patient37548/study7/...,Male,49,Frontal,AP,NaN,NaN,NaN,1.0,1.0,1.0,1.0,-1.0,NaN,0.0,1.0,NaN,NaN,1.0
189784,CheXpert-v1.0-small/train/patient45426/study5/...,Female,65,Frontal,AP,NaN,0.0,NaN,1.0,NaN,1.0,NaN,-1.0,-1.0,0.0,1.0,NaN,NaN,1.0
144069,CheXpert-v1.0-small/train/patient34615/study19...,Male,62,Frontal,AP,NaN,1.0,-1.0,1.0,NaN,1.0,1.0,-1.0,-1.0,0.0,1.0,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109698,CheXpert-v1.0-small/train/patient26341/study13...,Female,49,Frontal,AP,NaN,-1.0,NaN,1.0,NaN,-1.0,NaN,-1.0,1.0,0.0,-1.0,NaN,NaN,1.0
113382,CheXpert-v1.0-small/train/patient27192/study1/...,Male,68,Frontal,PA,NaN,0.0,NaN,1.0,NaN,NaN,-1.0,1.0,1.0,1.0,1.0,NaN,NaN,1.0
138084,CheXpert-v1.0-small/train/patient33155/study17...,Female,64,Frontal,AP,NaN,-1.0,NaN,1.0,NaN,-1.0,-1.0,NaN,1.0,1.0,1.0,NaN,NaN,1.0
